# Couchbase Analytics

## Setup

In [1]:
!pip3 install pandas

You should consider upgrading via the '/Users/yoshiyuki.kono/Documents/python_virtual/jupyter_analytics/bin/python -m pip install --upgrade pip' command.


In [2]:
# needed for any cluster connection
from couchbase.cluster import Cluster, ClusterOptions
from couchbase_core.cluster import PasswordAuthenticator

# needed to support SQL++ (N1QL) query
from couchbase.cluster import QueryOptions

In [3]:
from couchbase.exceptions import CouchbaseException
from couchbase.cluster import AnalyticsOptions

In [4]:
import pandas as pd
import json

In [5]:
user = 'Administrator'
password = 'couchbase'
host_url = 'couchbase://ec2-3-137-216-198.us-east-2.compute.amazonaws.com'

In [6]:
# get a reference to our cluster
cluster = Cluster(host_url, ClusterOptions(
  PasswordAuthenticator(user, password)))

## Data Service

In [7]:
# get a reference to our bucket
cb = cluster.bucket('travel-sample')
# get a reference to the default collection
cb_coll = cb.default_collection()

In [8]:
airline = {
  "type": "airline",
  "id": 8091,
  "callsign": "CBS",
  "iata": None,
  "icao": None,
  "name": "Couchbase Airways",
}

In [9]:
def upsert_document(doc):
  print("\nUpsert CAS: ")
  try:
    # key will equal: "airline_8091"
    key = doc["type"] + "_" + str(doc["id"])
    result = cb_coll.upsert(key, doc)
    print(result.cas)
  except Exception as e:
    print(e)
# get document function
def get_airline_by_key(key):
  print("\nGet Result: ")
  try:
    result = cb_coll.get(key)
    print(result.content_as[str])
  except Exception as e:
    print(e)

In [10]:
upsert_document(airline)


Upsert CAS: 
<Key='airline_8091', RC=0x419[LCB_ERR_UNKNOWN_HOST (1049)], Operational Error, Results=1, C Source=(src/multiresult.c,316), Context={'status_code': 4, 'opaque': 0, 'cas': 0, 'key': 'airline_8091', 'bucket': 'travel-sample', 'collection': '', 'scope': '', 'context': '', 'ref': '', 'endpoint': '', 'type': 'KVErrorContext'}, Tracing Output={"airline_8091": {"s": "kv:Unknown", "i": 5478264718919203149, "b": "travel-sample", "r": "", "t": 2500000, "debug_info": {"FILE": "src/callbacks.c", "FUNC": "dur_chain2", "LINE": 747}}}>


In [11]:
get_airline_by_key("airline_8091")


Get Result: 
<Key='airline_8091', RC=0x419[LCB_ERR_UNKNOWN_HOST (1049)], Operational Error, Results=1, C Source=(src/multiresult.c,316), Context={'status_code': 4, 'opaque': 1, 'cas': 0, 'key': 'airline_8091', 'bucket': 'travel-sample', 'collection': '', 'scope': '', 'context': '', 'ref': '', 'endpoint': '', 'type': 'KVErrorContext'}, Tracing Output={"airline_8091": {"s": "kv:Unknown", "i": 1416953737753764543, "b": "travel-sample", "r": "", "t": 2500000, "debug_info": {"FILE": "src/callbacks.c", "FUNC": "value_callback", "LINE": 852}}}>


In [12]:
# query for new document by callsign
def lookup_by_callsign(cs):
  print("\nLookup Result: ")
  try:
    sql_query = 'SELECT VALUE name FROM `travel-sample` WHERE type = "airline" AND callsign = $1'
    row_iter = cluster.query(
      sql_query,
      QueryOptions(positional_parameters=[cs]))
    for row in row_iter: print(row)
  except Exception as e:
    print(e)

In [13]:
lookup_by_callsign("CBS")


Lookup Result: 
<RC=0x419[LCB_ERR_UNKNOWN_HOST (1049)], HTTP Request failed. Examine 'objextra' for full result, Results=1, C Source=(src/pycbc_http.c,212), OBJ=ViewResult<rc=0x419[LCB_ERR_UNKNOWN_HOST (1049)], value=None, http_status=1049, tracing_context=0, tracing_output=None>, Context={'first_error_code': 0, 'http_response_code': 0, 'first_error_message': '', 'statement': 'SELECT VALUE name FROM `travel-sample` WHERE type = "airline" AND callsign = $1', 'client_context_id': 'f735e3d43d8d094b', 'query_params': '', 'http_response_body': '', 'endpoint': 'ip-10-0-0-249.us-east-2.compute.internal:8093', 'type': 'QueryErrorContext'}, Tracing Output={":nokey:0": null}>


## Analytics

In [14]:
try:
    result = cluster.analytics_query("select \"hello\" as greeting")

    for row in result.rows():
        print("Found row:")
        print(row)

    #print("Reported execution time: "
    #      + result.metaData().metrics().executionTime())
except CouchbaseException as ex:
    import traceback
    traceback.print_exc()

Found row:
{'greeting': 'hello'}


https://docs.couchbase.com/server/current/analytics/primer-beer.html

```
 CREATE DATASET breweries ON `beer-sample` WHERE `type` = "brewery";

 CREATE DATASET beers ON `beer-sample` WHERE `type` = "beer";
```
```
SELECT br.brewery_id, COUNT(*) AS num_beers
FROM beers br
/*+ hash */
GROUP BY br.brewery_id
HAVING COUNT(*) > 30
ORDER BY COUNT(*) DESC;
```
```
SELECT bw.name,
   COUNT(*) AS num_beers,
   AVG(br.abv) AS abv_avg,
   MIN(br.abv) AS abv_min,
   MAX(br.abv) AS abv_max
FROM breweries bw, beers br
WHERE br.brewery_id = meta(bw).id
GROUP BY bw.name
ORDER BY num_beers DESC
LIMIT 3;
```

In [15]:
query = "SELECT br.brewery_id, COUNT(*) AS num_beers FROM beers br GROUP BY br.brewery_id HAVING COUNT(*) > 30 ORDER BY COUNT(*) DESC;"

In [16]:
json_string = "{"
counter = 0
try:
    result = cluster.analytics_query(query)

    for row in result.rows():
        if counter != 0:
            json_string += ","
        counter += 1
        json_string += "'" + str(counter) + "':" + str(row)


    #print("Reported execution time: "
    #      + result.metaData().metrics().executionTime())
except CouchbaseException as ex:
    import traceback
    traceback.print_exc()
json_string += "}"

In [17]:
json_string

"{'1':{'num_beers': 57, 'brewery_id': 'midnight_sun_brewing_co'},'2':{'num_beers': 49, 'brewery_id': 'rogue_ales'},'3':{'num_beers': 38, 'brewery_id': 'anheuser_busch'},'4':{'num_beers': 37, 'brewery_id': 'egan_brewing'},'5':{'num_beers': 37, 'brewery_id': 'troegs_brewing'},'6':{'num_beers': 36, 'brewery_id': 'boston_beer_company'},'7':{'num_beers': 34, 'brewery_id': 'f_x_matt_brewing'},'8':{'num_beers': 34, 'brewery_id': 'titletown_brewing'},'9':{'num_beers': 33, 'brewery_id': 'sierra_nevada_brewing_co'},'10':{'num_beers': 32, 'brewery_id': 'stone_brewing_co'},'11':{'num_beers': 31, 'brewery_id': 'southern_tier_brewing_co'}}"

In [18]:
result.rows()

[{'num_beers': 57, 'brewery_id': 'midnight_sun_brewing_co'},
 {'num_beers': 49, 'brewery_id': 'rogue_ales'},
 {'num_beers': 38, 'brewery_id': 'anheuser_busch'},
 {'num_beers': 37, 'brewery_id': 'egan_brewing'},
 {'num_beers': 37, 'brewery_id': 'troegs_brewing'},
 {'num_beers': 36, 'brewery_id': 'boston_beer_company'},
 {'num_beers': 34, 'brewery_id': 'f_x_matt_brewing'},
 {'num_beers': 34, 'brewery_id': 'titletown_brewing'},
 {'num_beers': 33, 'brewery_id': 'sierra_nevada_brewing_co'},
 {'num_beers': 32, 'brewery_id': 'stone_brewing_co'},
 {'num_beers': 31, 'brewery_id': 'southern_tier_brewing_co'}]

In [19]:
s = pd.read_json(json_string.replace("'", '"'))

In [20]:
s.T

,num_beers,brewery_id
1,57,midnight_sun_brewing_co
2,49,rogue_ales
3,38,anheuser_busch
4,37,egan_brewing
5,37,troegs_brewing
6,36,boston_beer_company
7,34,f_x_matt_brewing
8,34,titletown_brewing
9,33,sierra_nevada_brewing_co
10,32,stone_brewing_co
